In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1. Carga los datos limpios
processed_dir = 'data/processed'
train = pd.read_csv(os.path.join(processed_dir, 'train.csv'))
test  = pd.read_csv(os.path.join(processed_dir, 'test.csv'))

# 2. Construye la matriz usuario×ítem en formato sparse
#    Restamos la media de cada usuario para centrar
user_means = train.groupby('user_id')['rating'].mean()
train['rating_centered'] = train.apply(lambda r: r.rating - user_means.loc[r.user_id], axis=1)
# Creamos un CSR matrix: filas=usuarios, columnas=libros
row = train['user_id'].values - 1
col = train['book_id'].values - 1
data = train['rating_centered'].values
R_csr = csr_matrix((data, (row, col)))

# 3. Entrena el modelo de vecinos
#    Usaremos similitud coseno (metric='cosine') y buscaremos k vecinos
k = 20
knn = NearestNeighbors(n_neighbors=k, metric='cosine', n_jobs=-1)
knn.fit(R_csr)

# 4. Función de predicción
def predict(u, i):
    """
    u: índice de usuario en base 0
    i: índice de ítem en base 0
    """
    # Si el usuario no tiene vecinos o el ítem nunca fue valorado
    if R_csr.shape[0] <= u or R_csr.shape[1] <= i or R_csr[:, i].nnz == 0:
        return user_means.get(u+1, train['rating'].mean())
    # Encuentra vecinos de u
    distances, neighbors = knn.kneighbors(R_csr[u], return_distance=True)
    sims = 1 - distances.flatten()          # coseno → similitud = 1 - distancia
    neigh_ids = neighbors.flatten()
    # Desviaciones de los vecinos para el ítem i
    devs = R_csr[neigh_ids, i].toarray().flatten()
    # Filtramos vecinos que realmente hayan valorado el ítem (devs ≠ 0)
    mask = devs != 0
    if not mask.any():
        return user_means.loc[u+1]
    sims, devs = sims[mask], devs[mask]
    # Predicción: media usuario + media ponderada de desviaciones
    pred = user_means.loc[u+1] + np.dot(sims, devs) / np.sum(sims)
    # Recorte al rango [1,5]
    return float(np.clip(pred, 1, 5))

# 5. Evaluación en test
y_true, y_pred = [], []
for _, row in test.iterrows():
    u = int(row.user_id) - 1
    i = int(row.book_id) - 1
    y_true.append(row.rating)
    y_pred.append(predict(u, i))

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae  = mean_absolute_error(y_true, y_pred)

print(f"KNN sklearn → RMSE: {rmse:.4f}, MAE: {mae:.4f}")


C:\Users\MAM\AppData\Local\Temp\ipykernel_888\4127417828.py:50: RuntimeWarning: invalid value encountered in scalar divide
  pred = user_means.loc[u+1] + np.dot(sims, devs) / np.sum(sims)
